In this notebook we will demonstrate using the fastText library to perform text classificatoin on the dbpedia data which can we downloaded from [here](https://github.com/le-scientifique/torchDatasets/raw/master/dbpedia_csv.tar.gz). <br>fastText is a library for learning of word embeddings and text classification created by Facebook's AI Research (FAIR) lab. The model allows to create an unsupervised learning or supervised learning algorithm for obtaining vector representations for words. Facebook makes available pretrained models for 294 languages(source: [wiki](https://en.wikipedia.org/wiki/FastText)).<br>
**Note**: This notebook uses an older version of fasttext.

In [2]:
import os
import pandas as pd
import wget
import tarfile

In [3]:
try :
    
    from google.colab import files
    
    # downloading the data
    !wget -P DATAPATH https://github.com/le-scientifique/torchDatasets/raw/master/dbpedia_csv.tar.gz

    # untaring the required file
    !tar -xvf DATAPATH/dbpedia_csv.tar.gz -C DATAPATH

    # sneek peek in the folder structure
    !ls -lah DATAPATH
    
    # specifying the data_path
    data_path = 'DATAPATH'
    
except ModuleNotFoundError:
    
    if not os.path.exists(os.getcwd()+'\\Data\\dbpedia_csv') :
        # downloading the data
        url="https://github.com/le-scientifique/torchDatasets/raw/master/dbpedia_csv.tar.gz"
        path=os.getcwd()+'\Data'
        wget.download(url,path)

        # untaring the required file
        temp=path+'\dbpedia_csv.tar.gz'
        tar = tarfile.open(temp, "r:gz")
        tar.extractall(path)     
        tar.close()
    
    # specifying the data_path
    data_path='Data'

In [4]:
# Loading train data
train_file = data_path + '/dbpedia_csv/train.csv'
df = pd.read_csv(train_file, header=None, names=['class','name','description'])
# Loading test data
test_file = data_path + '/dbpedia_csv/test.csv'
df_test = pd.read_csv(test_file, header=None, names=['class','name','description'])
# Data we have
print("Train:{} Test:{}".format(df.shape,df_test.shape))


Train:(560000, 3) Test:(70000, 3)


In [5]:
# Since we have no clue about the classes lets build one
# Mapping from class number to class name
class_dict={
            1:'Company',
            2:'EducationalInstitution',
            3:'Artist',
            4:'Athlete',
            5:'OfficeHolder',
            6:'MeanOfTransportation',
            7:'Building',
            8:'NaturalPlace',
            9:'Village',
            10:'Animal',
            11:'Plant',
            12:'Album',
            13:'Film',
            14:'WrittenWork'
        }

# Mapping the classes
df['class_name'] = df['class'].map(class_dict)
df.head()

,class,name,description,class_name
0,1,E. D. Abbott Ltd,Abbott of Farnham E D Abbott Limited was a Br...,Company
1,1,Schwan-Stabilo,Schwan-STABILO is a German maker of pens for ...,Company
2,1,Q-workshop,Q-workshop is a Polish company located in Poz...,Company
3,1,Marvell Software Solutions Israel,Marvell Software Solutions Israel known as RA...,Company
4,1,Bergan Mercy Medical Center,Bergan Mercy Medical Center is a hospital loc...,Company


In [6]:
df["class_name"].value_counts()

Company                   40000
EducationalInstitution    40000
Artist                    40000
Athlete                   40000
OfficeHolder              40000
MeanOfTransportation      40000
Building                  40000
NaturalPlace              40000
Village                   40000
Animal                    40000
Plant                     40000
Album                     40000
Film                      40000
WrittenWork               40000
Name: class_name, dtype: int64

In [7]:
# Lets do some cleaning of this text
def clean_it(text, normalize=True):
    # Replacing possible issues with data. We can add or reduce the replacement in this chain
    s = str(text).replace(',',' ').replace('"','').replace('\'',' \' ').replace('.',' . ').replace('(',' ( ').\
            replace(')',' ) ').replace('!',' ! ').replace('?',' ? ').replace(':',' ').replace(';',' ').lower()
    
    # normalizing / encoding the text
    if normalize:
        # This is to do with unicode compatibility.  See: https://docs.python.org/3/library/unicodedata.html
        s = s.normalize('NFKD').str.encode('ascii','ignore').str.decode('utf-8')
    
    return s

# Now lets define a small function where we can use above cleaning on datasets
def clean_df(data, cleanit=False, shuffleit=False, encodeit=False, label_prefix='__class__'):
    # Defining the new data
    df = data[['name','description']].copy(deep=True)
    df['class'] = label_prefix + data['class'].astype(str) + ' '
    
    # cleaning it
    if cleanit:
        df['name'] = df['name'].apply(lambda x: clean_it(x, encodeit))
        df['description'] = df['description'].apply(lambda x: clean_it(x, encodeit))
    
    # shuffling it
    if shuffleit:
        df.sample(frac=1).reset_index(drop=True)
            
    return df

In [8]:
%%time
# Transform the datasets using the above clean functions
df_train_cleaned = clean_df(df, True, True)
df_test_cleaned = clean_df(df_test, True, True)

CPU times: total: 2.78 s
Wall time: 2.78 s


In [9]:
# Write files to disk as fastText classifier API reads files from disk.
train_file = data_path + '/dbpedia_train.csv'
df_train_cleaned.to_csv(train_file, header=None, index=False, columns=['class','name','description'] )

test_file = data_path + '/dbpedia_test.csv'
df_test_cleaned.to_csv(test_file, header=None, index=False, columns=['class','name','description'] )


Now that we have the train and test files written into disk in a format fastText wants, we are ready to use it for text classification!

In [10]:
%%time
## Using fastText for feature extraction and training
from fasttext import train_supervised 
"""fastText expects and training file (csv), a model name as input arguments.
label_prefix refers to the prefix before label string in the dataset.
default is __label__. In our dataset, it is __class__. 
There are several other parameters which can be seen in: 
https://pypi.org/project/fasttext/
"""
model = train_supervised(input=train_file, label="__class__", lr=1.0, epoch=75, loss='ova', wordNgrams=2, dim=200, thread=4, verbose=100)

CPU times: total: 59min 15s
Wall time: 14min 53s


In [11]:
for k in range(1,6):
    results = model.test(test_file,k=k)
    print(f"Test Samples: {results[0]} Precision@{k} : {results[1]*100:2.4f} Recall@{k} : {results[2]*100:2.4f}")

Test Samples: 70000 Precision@1 : 98.3700 Recall@1 : 98.3700
Test Samples: 70000 Precision@2 : 49.5686 Recall@2 : 99.1371
Test Samples: 70000 Precision@3 : 33.0624 Recall@3 : 99.1871
Test Samples: 70000 Precision@4 : 24.8139 Recall@4 : 99.2557
Test Samples: 70000 Precision@5 : 19.8740 Recall@5 : 99.3700


Try training a classifier on this dataset with, say, LogisticRegression to realize how fast fastText is! 93% Precision and Recall are hard numbers to beat, too!

In [12]:
df_test_cleaned.head()

,name,description,class
0,ty ku,ty ku /taɪkuː/ is an american alcoholic bever...,__class__1
1,odd lot entertainment,oddlot entertainment founded in 2001 by longt...,__class__1
2,henkel,henkel ag & company kgaa operates worldwide w...,__class__1
3,goat store,the goat store ( games of all type store ) ...,__class__1
4,ragwing aircraft designs,ragwing aircraft designs ( also called the r...,__class__1
